<a href="https://colab.research.google.com/github/celikn/GEE_Repository/blob/main/Raster_Data_Modification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rasterio
!pip install fiona

Verilen iki raster path bilgisi ile ndvi hesabi




In [ ]:
import rasterio
from rasterio import plot
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import os

def createIndexImage(bandPaths,outputPath,indexType,showImage,plotAfterProcess):
        if indexType=="NDVI":
           band_a = rasterio.open(bandPaths[0]) #red
           band_b= rasterio.open(bandPaths[1]) #red
           print(f"First Band weigth/width: {band_a.height},{band_a.width}")
           print(f"First Crs : {band_a.crs}")

           #generate nir and red objects as arrays in float64 format
           red = band_a.read(1).astype('float64')
           nir = band_b.read(1).astype('float64')
           #ndvi calculation, empty cells or nodata cells are reported as 0
           ndvi=np.where(
              (nir+red)==0., 
              0, 
              (nir-red)/(nir+red))
           
           ndviImage = rasterio.open(outputPath,'w',driver='Gtiff',
                          width=band_a.width, 
                          height = band_a.height, 
                          count=1, crs=band_a.crs, 
                          transform=band_a.transform, 
                          dtype='float64')
           ndviImage.write(ndvi,1)
           ndviImage.close()     
           

        if showImage==True:
          fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
          plot.show(band_a, ax=ax1, cmap='Blues') #red
          plot.show(band_b, ax=ax2, cmap='Blues') #nir
          fig.tight_layout()

        if plotAfterProcess==True:
          ndvi = rasterio.open(outputPath)
          fig = plt.figure(figsize=(18,12))
          plot.show(ndvi)
        
indexType="NDVI"
showImage=False
plotAfterProcess=False
bandPaths=['/content/torchgeotest/Onera Satellite Change Detection dataset - Images/cupertino/imgs_1_rect/B04.tif','/content/torchgeotest/Onera Satellite Change Detection dataset - Images/cupertino/imgs_1_rect/B08.tif']
outputPath="/content/ndviImage.tif"

createIndexImage(bandPaths,outputPath,indexType="NDVI",showImage=True,plotAfterProcess=True)




Multiband raster verisinin her bir bandindan ayri bir tif olusturma

In [ ]:
## Creating Seperate Bands 
bands = {
    "B1": "Coastal Aerosol",
    "B2": "Blue",
    "B3": "Green",
    "B4": "Red",
    "B5": "Vegetation Red Edge 1",
    "B6": "Vegetation Red Edge 2",
    "B7": "Vegetation Red Edge 3",
    "B8": "NIR 1",
    "B8A": "NIR 2",
    "B9": "Water Vapour",
    "B10": "SWIR 1",
    "B11": "SWIR 2",
    "B12": "SWIR 3"
}

import os
import rasterio

out_folder ="/content/drive/MyDrive/SentinelImages_Ankara_Clipped_2016/All_Bands"
input_tif ="/content/drive/MyDrive/SentinelImages_Ankara_Clipped_2016/S2SR_20160929.tif"
#out_folder ="/content/drive/MyDrive/SentinelImages_Ankara_Clipped_2020/All_Bands"
#input_tif ="/content/drive/MyDrive/SentinelImages_Ankara_Clipped_2020/S2SR_20200928.tif"
src_raster = rasterio.open(input_tif)
for band in range(1, src_raster.count + 1):
        single_band = src_raster.read(band)

        # get the output name
        out_name = os.path.basename(input_tif)
        file, ext = os.path.splitext(out_name)
        name = file + "_" + "B" + str(band) + ".tif"
        out_img = os.path.join(out_folder, name)

        print(out_img + " done")

        # Copy the metadata
        out_meta = src_raster.meta.copy()

        out_meta.update({"count": 1})

        # save the clipped raster to disk
        with rasterio.open(out_img, "w", **out_meta) as dest:
               dest.write(single_band, 1)



Raster bounding box verisini geojson olarak olusturma


In [ ]:
# imports
from shapely.geometry import box, mapping
import fiona
import rasterio

out_folder ="/content/drive/MyDrive/SentinelImages_Ankara_Clipped_2016/All_Bands"
input_tif ="/content/drive/MyDrive/SentinelImages_Ankara_Clipped_2016/S2SR_20160929.tif"
#out_folder ="/content/drive/MyDrive/SentinelImages_Ankara_Clipped_2020/All_Bands"
#input_tif ="/content/drive/MyDrive/SentinelImages_Ankara_Clipped_2020/S2SR_20200928.tif"
raster = rasterio.open(input_tif)
output_fileName='/content/drive/MyDrive/SentinelImages_Ankara_Clipped_2016/ankara.geojson'
#output_fileName='/content/drive/MyDrive/SentinelImages_Ankara_Clipped_2020/ankara.geojson'

# create a Polygon from the raster bounds
bbox = box(*raster.bounds)

# create a schema with no properties
schema = {'geometry': 'Polygon', 'properties': {}}

# create shapefile
#with fiona.open(output_fileName, 'w', driver='ESRI Shapefile',
#                crs=raster.crs.to_dict(), schema=schema) as c:
#    c.write({'geometry': mapping(bbox), 'properties': {}})


with fiona.open(output_fileName,'w',  driver ="GeoJSON", schema=schema, encoding = 'utf-8', crs=raster.crs.to_dict()) as geojson:
     geojson.write({'geometry': mapping(bbox), 'properties': {}})